In [596]:
import pandas as pd
from datetime import datetime
from plyer import notification
import requests
import numpy as np
import sqlite3
import json

In [597]:
conn = sqlite3.connect('coderhouse.db')

Função responsável por salvar o DataFrame no Banco de dados local.

In [598]:
def salva_bd(df, nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    df.to_sql(nome_tabela,conn,if_exists='replace',index = False)
    conn.close()
    return None

Função responsável por carregar o DataFrame do banco de dados local.

In [599]:
def carrega_bd(nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    query = f"select * from {nome_tabela}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

Função responsável por listar todas as tabelas salvas no banco de dados local.

In [600]:
def lista_tabelas():
    conn = sqlite3.connect('coderhouse.db')
    query = "select name from sqlite_master where type='table'"
    schema = pd.read_sql(query, conn)
    conn.close()
    return schema

In [601]:
urls = [ 
    'https://brasilapi.com.br/api/banks/v1', #bancos ok
    'https://brasilapi.com.br/api/cvm/corretoras/v1', #corretoras ok
    'https://restcountries.com/v3.1/all' #rest countries ok
]

Função responsável por emitir um alerta caso a requisição da API retorne falha.

In [602]:
def Alerta(nivel, base, etapa):
    
    data_atual = datetime.now()
    data_formatada = data_atual.strftime('%d/%m/%d')
    mensagem = (f'Falha no carregamento da base {base} na etapa {etapa} em {data_formatada}')
    title = nivel
    notification.notify(title = title, message= mensagem, timeout= 10)

Função responsável por direcionar as APIs em seus respectivos Data Frames.

In [603]:
def Extraçao():

    global df_bancos, df_corretoras, df_resties

    for url in urls:
        request = requests.get(url)
        response_code = request.status_code
        if response_code == 200:
            if url == 'https://brasilapi.com.br/api/banks/v1':
                df_bancos = pd.DataFrame(request.json())
                
            elif url == 'https://brasilapi.com.br/api/cvm/corretoras/v1':
                df_corretoras = pd.DataFrame(request.json())
                
            elif url == 'https://restcountries.com/v3.1/all':
                df_resties = pd.DataFrame(request.json())
                
            else:
                print('URL fora da base de dados.')
        else:
            nivel = 'ATENÇÃO: Erro Grave'
            Alerta(nivel = nivel, base = 'APIs', etapa = 'Extraçao')

In [604]:
Extraçao()

                                           INICIANDO TRATAMENTO DO DF_BANCOS

In [605]:
df_bancos.head(2)

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.


In [606]:
df_bancos_mod = df_bancos.copy() #mantendo o df original sem que o mesmo seja alterado pelas modificações posteriores

In [607]:
df_bancos_mod.head(2)

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.


In [608]:
df_bancos_mod['code'].fillna(0, inplace=True)

In [609]:
df_bancos_mod['code'] = df_bancos_mod['code'].astype(int)

In [610]:
colunas_com_nan = ['name', 'fullName']

In [611]:
df_bancos_mod = df_bancos_mod.dropna(subset=colunas_com_nan)

In [612]:
df_bancos_mod['fullName'] = df_bancos_mod['fullName'].str.upper()

In [613]:
df_bancos_mod['name'] = df_bancos_mod['name'].str.upper()

In [614]:
df_bancos_mod.rename(columns={'ispb': 'ISPB'}, inplace=True)
df_bancos_mod.rename(columns={'name': 'SOCIAL'}, inplace=True)
df_bancos_mod.rename(columns={'code': 'CÓDIGO'}, inplace=True)
df_bancos_mod.rename(columns={'fullName': 'RAZÃO SOCIAL'}, inplace=True)

In [615]:
df_bancos_mod.head(4)

,ISPB,SOCIAL,CÓDIGO,RAZÃO SOCIAL
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.
2,00038121,SELIC,0,BANCO CENTRAL DO BRASIL - SELIC
3,00038166,BACEN,0,BANCO CENTRAL DO BRASIL


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [616]:
salva_bd(df_bancos_mod,'bancos')

In [617]:
lista_tabelas()

,name
0,produtos
1,restaurantes
2,fastfood
3,corretoras
4,resties
5,bancos


In [618]:
bancos_teste = carrega_bd('bancos')

In [619]:
bancos_teste.head(2)

,ISPB,SOCIAL,CÓDIGO,RAZÃO SOCIAL
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.


                                       INICIANDO TRATAMENTO DO DF_CORRETORAS

In [620]:
df_corretoras.head(2)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14


In [621]:
df_corretoras_mod = df_corretoras.copy()

In [622]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      372 non-null    object
 1   type                      372 non-null    object
 2   nome_social               372 non-null    object
 3   nome_comercial            372 non-null    object
 4   status                    372 non-null    object
 5   email                     372 non-null    object
 6   telefone                  372 non-null    object
 7   cep                       372 non-null    object
 8   pais                      372 non-null    object
 9   uf                        372 non-null    object
 10  municipio                 372 non-null    object
 11  bairro                    372 non-null    object
 12  complemento               372 non-null    object
 13  logradouro                372 non-null    object
 14  data_patrimonio_liquido   

In [623]:
df_corretoras_mod['data_patrimonio_liquido']

0      2005-12-31
1      2002-12-31
2      1989-12-31
3      2022-12-31
4      2000-12-31
          ...    
367    2017-12-31
368    2017-12-31
369    2022-12-31
370    1996-12-31
371    2022-12-31
Name: data_patrimonio_liquido, Length: 372, dtype: object

In [624]:
df_corretoras_mod['data_patrimonio_liquido'] = pd.to_datetime(df_corretoras_mod['data_patrimonio_liquido'])

In [625]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   cnpj                      372 non-null    object        
 1   type                      372 non-null    object        
 2   nome_social               372 non-null    object        
 3   nome_comercial            372 non-null    object        
 4   status                    372 non-null    object        
 5   email                     372 non-null    object        
 6   telefone                  372 non-null    object        
 7   cep                       372 non-null    object        
 8   pais                      372 non-null    object        
 9   uf                        372 non-null    object        
 10  municipio                 372 non-null    object        
 11  bairro                    372 non-null    object        
 12  complemento           

In [626]:
df_corretoras_mod['data_patrimonio_liquido'] = df_corretoras_mod['data_patrimonio_liquido'].dt.strftime('%d/%m/%Y')

In [627]:
df_corretoras_mod['data_inicio_situacao']

0      2006-10-05
1      2002-10-14
2      1990-06-12
3      2022-06-20
4      2001-04-11
          ...    
367    2018-08-27
368    2019-09-24
369    1967-12-06
370    1997-02-21
371    1998-02-10
Name: data_inicio_situacao, Length: 372, dtype: object

In [628]:
df_corretoras_mod['data_inicio_situacao'] = pd.to_datetime(df_corretoras_mod['data_inicio_situacao'])

In [629]:
df_corretoras_mod['data_inicio_situacao'] = df_corretoras_mod['data_inicio_situacao'].dt.strftime('%d/%m/%Y')

In [630]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      372 non-null    object
 1   type                      372 non-null    object
 2   nome_social               372 non-null    object
 3   nome_comercial            372 non-null    object
 4   status                    372 non-null    object
 5   email                     372 non-null    object
 6   telefone                  372 non-null    object
 7   cep                       372 non-null    object
 8   pais                      372 non-null    object
 9   uf                        372 non-null    object
 10  municipio                 372 non-null    object
 11  bairro                    372 non-null    object
 12  complemento               372 non-null    object
 13  logradouro                372 non-null    object
 14  data_patrimonio_liquido   

In [631]:
df_corretoras_mod['data_registro'] = pd.to_datetime(df_corretoras_mod['data_registro'])

In [632]:
df_corretoras_mod['data_registro'] = df_corretoras_mod['data_registro'].dt.strftime('%d/%m/%Y') 

In [633]:
df_corretoras_mod.head(2)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


In [634]:
df_corretoras_mod.columns = [col.capitalize() for col in df_corretoras.columns]

In [635]:
df_corretoras_mod.rename(columns={'Cnpj': 'CNPJ'}, inplace=True)

In [636]:
df_corretoras_mod.rename(columns={'Uf': 'UF'}, inplace=True)

In [637]:
df_corretoras_mod.rename(columns={'Cep': 'CEP'}, inplace=True)

In [638]:
df_corretoras_mod.rename(columns={'Email': 'E-mail'}, inplace=True)

In [639]:
df_corretoras_mod.rename(columns={'Pais': 'País'}, inplace=True)

In [640]:
df_corretoras_mod.rename(columns={'Type': 'Tipo'}, inplace=True)

In [641]:
df_corretoras_mod.head(2)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


In [642]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CNPJ                      372 non-null    object
 1   Tipo                      372 non-null    object
 2   Nome_social               372 non-null    object
 3   Nome_comercial            372 non-null    object
 4   Status                    372 non-null    object
 5   E-mail                    372 non-null    object
 6   Telefone                  372 non-null    object
 7   CEP                       372 non-null    object
 8   País                      372 non-null    object
 9   UF                        372 non-null    object
 10  Municipio                 372 non-null    object
 11  Bairro                    372 non-null    object
 12  Complemento               372 non-null    object
 13  Logradouro                372 non-null    object
 14  Data_patrimonio_liquido   

In [643]:
linhas_com_missing = df_corretoras_mod[df_corretoras_mod.isna().any(axis=1)]

In [644]:
linhas_com_missing

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
18,99988065000154,CORRETORAS,ALVES FERREIRA CV LTDA,,CANCELADA,,,80020,BRASIL,PR,CURITIBA,,,"RUA MAL.DEODORO, 235 - 2 ANDAR",NaN,,2747,29/04/1993,27/12/1991
70,12865507000197,CORRETORAS,"BRICKELL S.A. CRÉDITO, FINANCIAMENTO E INVESTI...",--,CANCELADA,brickellcfi@brickellcfi.com.br,21989735,4534002,BRASIL,SP,SÃO PAULO,ITAIM BIBI,SALA 1302 E1303,"R JOAQUIM FLORIANO, 466",NaN,,3492,23/01/1995,28/03/1988
228,36864992000142,CORRETORAS,MAF DISTRIBUIDORA DE TITULOS E VALORES MOBILIA...,MAF DTVM,CANCELADA,juridico.admfiduciaria@apexgroup.com,45601000,22250040,,RJ,RIO DE JANEIRO,BOTAFOGO,TORRE PÃO DE AÇUCAR,"PRAIA DE BOTAFOGO, 501/ 5º ANDAR PARTE",NaN,,3751,06/11/2020,06/11/2020


In [645]:
df_corretoras_mod = df_corretoras_mod.dropna()

In [646]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 369 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CNPJ                      369 non-null    object
 1   Tipo                      369 non-null    object
 2   Nome_social               369 non-null    object
 3   Nome_comercial            369 non-null    object
 4   Status                    369 non-null    object
 5   E-mail                    369 non-null    object
 6   Telefone                  369 non-null    object
 7   CEP                       369 non-null    object
 8   País                      369 non-null    object
 9   UF                        369 non-null    object
 10  Municipio                 369 non-null    object
 11  Bairro                    369 non-null    object
 12  Complemento               369 non-null    object
 13  Logradouro                369 non-null    object
 14  Data_patrimonio_liquido   369 n

In [647]:
df_corretoras_mod.head(5)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,31/12/1989,5995252.29,329,12/06/1990,08/07/1986
3,44527444000155,CORRETORAS,ABN AMRO CORRETORA DE T-TULOS E VALORES MOBILI...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",31/12/2022,3729687.41,3794,20/06/2022,20/06/2022
4,33819590000138,CORRETORAS,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,,5095755,20040040,BRASIL,RJ,RIO DE JANEIRO,CENTRO,,"PRAÇA OLAVO BILAC, 28/ SL. 608",31/12/2000,291862.38,1112,11/04/2001,19/04/1968


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [648]:
salva_bd(df_corretoras_mod,'corretoras')

In [649]:
lista_tabelas()

,name
0,produtos
1,restaurantes
2,fastfood
3,resties
4,bancos
5,corretoras


In [650]:
corretora_teste = carrega_bd('corretoras')

In [651]:
corretora_teste.head(2)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


                                         INICIANDO TRATAMENTO DO DF_RESTIES

In [652]:
df_resties.head(2)

,name,tld,cca2,ccn3,cca3,cioc,independent,status,unMember,currencies,idd,capital,altSpellings,region,subregion,languages,translations,latlng,landlocked,borders,area,demonyms,flag,maps,population,gini,fifa,car,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,postalCode
0,"{'common': 'Palestine', 'official': 'State of ...","[.ps, فلسطين.]",PS,275,PSE,PLE,False,officially-assigned,False,"{'EGP': {'name': 'Egyptian pound', 'symbol': '...","{'root': '+9', 'suffixes': ['70']}","[Ramallah, Jerusalem]","[PS, Palestine, State of, State of Palestine, ...",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة فلسطين', 'common': ...","[31.9, 35.2]",False,"[ISR, EGY, JOR]",6220.0,"{'eng': {'f': 'Palestinian', 'm': 'Palestinian...",🇵🇸,{'googleMaps': 'https://goo.gl/maps/QvTbkRdmdW...,4803269,{'2016': 33.7},PLE,"{'signs': ['PS'], 'side': 'right'}",[UTC+02:00],[Asia],"{'png': 'https://flagcdn.com/w320/ps.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [31.9, 35.2]}",NaN
1,"{'common': 'Kuwait', 'official': 'State of Kuw...",[.kw],KW,414,KWT,KUW,True,officially-assigned,True,"{'KWD': {'name': 'Kuwaiti dinar', 'symbol': 'د...","{'root': '+9', 'suffixes': ['65']}",[Kuwait City],"[KW, State of Kuwait, Dawlat al-Kuwait]",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة الكويت', 'common': ...","[29.5, 45.75]",False,"[IRQ, SAU]",17818.0,"{'eng': {'f': 'Kuwaiti', 'm': 'Kuwaiti'}, 'fra...",🇰🇼,{'googleMaps': 'https://goo.gl/maps/aqr3aNQjS1...,4270563,NaN,KUW,"{'signs': ['KWT'], 'side': 'right'}",[UTC+03:00],[Asia],"{'png': 'https://flagcdn.com/w320/kw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [29.37, 47.97]}","{'format': '#####', 'regex': '^(\d{5})$'}"


In [653]:
df_resties_mod = df_resties.copy()

In [654]:
df_resties_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          250 non-null    object 
 1   tld           249 non-null    object 
 2   cca2          250 non-null    object 
 3   ccn3          249 non-null    object 
 4   cca3          250 non-null    object 
 5   cioc          206 non-null    object 
 6   independent   249 non-null    object 
 7   status        250 non-null    object 
 8   unMember      250 non-null    bool   
 9   currencies    247 non-null    object 
 10  idd           250 non-null    object 
 11  capital       246 non-null    object 
 12  altSpellings  250 non-null    object 
 13  region        250 non-null    object 
 14  subregion     245 non-null    object 
 15  languages     249 non-null    object 
 16  translations  250 non-null    object 
 17  latlng        250 non-null    object 
 18  landlocked    250 non-null    

In [655]:
valores_ausentes_por_coluna = df_resties_mod.isna().sum()

In [656]:
print(valores_ausentes_por_coluna)

name              0
tld               1
cca2              0
ccn3              1
cca3              0
cioc             44
independent       1
status            0
unMember          0
currencies        3
idd               0
capital           4
altSpellings      0
region            0
subregion         5
languages         1
translations      0
latlng            0
landlocked        0
borders          85
area              0
demonyms          1
flag              0
maps              0
population        0
gini             83
fifa             44
car               0
timezones         0
continents        0
flags             0
coatOfArms        0
startOfWeek       0
capitalInfo       0
postalCode      101
dtype: int64


In [657]:
df_resties_mod.columns = [col.capitalize() for col in df_resties_mod.columns]

In [658]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Cioc,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Borders,Area,Demonyms,Flag,Maps,Population,Gini,Fifa,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Postalcode
0,"{'common': 'Palestine', 'official': 'State of ...","[.ps, فلسطين.]",PS,275,PSE,PLE,False,officially-assigned,False,"{'EGP': {'name': 'Egyptian pound', 'symbol': '...","{'root': '+9', 'suffixes': ['70']}","[Ramallah, Jerusalem]","[PS, Palestine, State of, State of Palestine, ...",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة فلسطين', 'common': ...","[31.9, 35.2]",False,"[ISR, EGY, JOR]",6220.0,"{'eng': {'f': 'Palestinian', 'm': 'Palestinian...",🇵🇸,{'googleMaps': 'https://goo.gl/maps/QvTbkRdmdW...,4803269,{'2016': 33.7},PLE,"{'signs': ['PS'], 'side': 'right'}",[UTC+02:00],[Asia],"{'png': 'https://flagcdn.com/w320/ps.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [31.9, 35.2]}",NaN
1,"{'common': 'Kuwait', 'official': 'State of Kuw...",[.kw],KW,414,KWT,KUW,True,officially-assigned,True,"{'KWD': {'name': 'Kuwaiti dinar', 'symbol': 'د...","{'root': '+9', 'suffixes': ['65']}",[Kuwait City],"[KW, State of Kuwait, Dawlat al-Kuwait]",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة الكويت', 'common': ...","[29.5, 45.75]",False,"[IRQ, SAU]",17818.0,"{'eng': {'f': 'Kuwaiti', 'm': 'Kuwaiti'}, 'fra...",🇰🇼,{'googleMaps': 'https://goo.gl/maps/aqr3aNQjS1...,4270563,NaN,KUW,"{'signs': ['KWT'], 'side': 'right'}",[UTC+03:00],[Asia],"{'png': 'https://flagcdn.com/w320/kw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [29.37, 47.97]}","{'format': '#####', 'regex': '^(\d{5})$'}"


In [659]:
df_resties_mod['Name'] = [pais['common']for pais in df_resties_mod['Name']]

In [660]:
df_resties_mod['Postalcode']

0                                                    NaN
1              {'format': '#####', 'regex': '^(\d{5})$'}
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
245                                                  NaN
246    {'format': '@# #@@|@## #@@|@@# #@@|@@## #@@|@#...
247             {'format': '####', 'regex': '^(\d{4})$'}
248                                                  NaN
249             {'format': '####', 'regex': '^(\d{4})$'}
Name: Postalcode, Length: 250, dtype: object

In [661]:
df_resties_mod = df_resties_mod.drop('Postalcode', axis=1)

In [662]:
pd.options.display.max_columns = None

In [663]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Cioc,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Borders,Area,Demonyms,Flag,Maps,Population,Gini,Fifa,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo
0,Palestine,"[.ps, فلسطين.]",PS,275,PSE,PLE,False,officially-assigned,False,"{'EGP': {'name': 'Egyptian pound', 'symbol': '...","{'root': '+9', 'suffixes': ['70']}","[Ramallah, Jerusalem]","[PS, Palestine, State of, State of Palestine, ...",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة فلسطين', 'common': ...","[31.9, 35.2]",False,"[ISR, EGY, JOR]",6220.0,"{'eng': {'f': 'Palestinian', 'm': 'Palestinian...",🇵🇸,{'googleMaps': 'https://goo.gl/maps/QvTbkRdmdW...,4803269,{'2016': 33.7},PLE,"{'signs': ['PS'], 'side': 'right'}",[UTC+02:00],[Asia],"{'png': 'https://flagcdn.com/w320/ps.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [31.9, 35.2]}"
1,Kuwait,[.kw],KW,414,KWT,KUW,True,officially-assigned,True,"{'KWD': {'name': 'Kuwaiti dinar', 'symbol': 'د...","{'root': '+9', 'suffixes': ['65']}",[Kuwait City],"[KW, State of Kuwait, Dawlat al-Kuwait]",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة الكويت', 'common': ...","[29.5, 45.75]",False,"[IRQ, SAU]",17818.0,"{'eng': {'f': 'Kuwaiti', 'm': 'Kuwaiti'}, 'fra...",🇰🇼,{'googleMaps': 'https://goo.gl/maps/aqr3aNQjS1...,4270563,NaN,KUW,"{'signs': ['KWT'], 'side': 'right'}",[UTC+03:00],[Asia],"{'png': 'https://flagcdn.com/w320/kw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [29.37, 47.97]}"


In [664]:
df_resties_mod['Currencies']

0      {'EGP': {'name': 'Egyptian pound', 'symbol': '...
1      {'KWD': {'name': 'Kuwaiti dinar', 'symbol': 'د...
2      {'GNF': {'name': 'Guinean franc', 'symbol': 'F...
3      {'NZD': {'name': 'New Zealand dollar', 'symbol...
4      {'AFN': {'name': 'Afghan afghani', 'symbol': '...
                             ...                        
245             {'EUR': {'name': 'Euro', 'symbol': '€'}}
246    {'GBP': {'name': 'British pound', 'symbol': '£...
247    {'NZD': {'name': 'New Zealand dollar', 'symbol...
248    {'STN': {'name': 'São Tomé and Príncipe dobra'...
249          {'MKD': {'name': 'denar', 'symbol': 'den'}}
Name: Currencies, Length: 250, dtype: object

In [665]:
df_resties_mod.head(1)

,Name,Tld,Cca2,Ccn3,Cca3,Cioc,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Borders,Area,Demonyms,Flag,Maps,Population,Gini,Fifa,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo
0,Palestine,"[.ps, فلسطين.]",PS,275,PSE,PLE,False,officially-assigned,False,"{'EGP': {'name': 'Egyptian pound', 'symbol': '...","{'root': '+9', 'suffixes': ['70']}","[Ramallah, Jerusalem]","[PS, Palestine, State of, State of Palestine, ...",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة فلسطين', 'common': ...","[31.9, 35.2]",False,"[ISR, EGY, JOR]",6220.0,"{'eng': {'f': 'Palestinian', 'm': 'Palestinian...",🇵🇸,{'googleMaps': 'https://goo.gl/maps/QvTbkRdmdW...,4803269,{'2016': 33.7},PLE,"{'signs': ['PS'], 'side': 'right'}",[UTC+02:00],[Asia],"{'png': 'https://flagcdn.com/w320/ps.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [31.9, 35.2]}"


In [666]:
df_resties_mod['Currencies'].dropna()

0      {'EGP': {'name': 'Egyptian pound', 'symbol': '...
1      {'KWD': {'name': 'Kuwaiti dinar', 'symbol': 'د...
2      {'GNF': {'name': 'Guinean franc', 'symbol': 'F...
3      {'NZD': {'name': 'New Zealand dollar', 'symbol...
4      {'AFN': {'name': 'Afghan afghani', 'symbol': '...
                             ...                        
245             {'EUR': {'name': 'Euro', 'symbol': '€'}}
246    {'GBP': {'name': 'British pound', 'symbol': '£...
247    {'NZD': {'name': 'New Zealand dollar', 'symbol...
248    {'STN': {'name': 'São Tomé and Príncipe dobra'...
249          {'MKD': {'name': 'denar', 'symbol': 'den'}}
Name: Currencies, Length: 247, dtype: object

In [667]:
# Função para extrair a chave 'name' do dicionário
def extrair_nome_moeda(dicionario):
    if isinstance(dicionario, dict):
        for chave, valor in dicionario.items():
            if 'name' in valor:
                return valor['name']
    return None
        

# Extrair e atribuir a chave 'name' à coluna 'currencies'
df_resties_mod['Currencies'] = df_resties_mod['Currencies'].apply(extrair_nome_moeda)

# Renomear a coluna para 'Currencies_Name'
df_resties_mod = df_resties_mod.rename(columns={'Currencies': 'Currencies_Name'})


In [668]:
df_resties_mod.head(1)

,Name,Tld,Cca2,Ccn3,Cca3,Cioc,Independent,Status,Unmember,Currencies_Name,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Borders,Area,Demonyms,Flag,Maps,Population,Gini,Fifa,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo
0,Palestine,"[.ps, فلسطين.]",PS,275,PSE,PLE,False,officially-assigned,False,Egyptian pound,"{'root': '+9', 'suffixes': ['70']}","[Ramallah, Jerusalem]","[PS, Palestine, State of, State of Palestine, ...",Asia,Western Asia,{'ara': 'Arabic'},"{'ara': {'official': 'دولة فلسطين', 'common': ...","[31.9, 35.2]",False,"[ISR, EGY, JOR]",6220.0,"{'eng': {'f': 'Palestinian', 'm': 'Palestinian...",🇵🇸,{'googleMaps': 'https://goo.gl/maps/QvTbkRdmdW...,4803269,{'2016': 33.7},PLE,"{'signs': ['PS'], 'side': 'right'}",[UTC+02:00],[Asia],"{'png': 'https://flagcdn.com/w320/ps.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [31.9, 35.2]}"


In [669]:
df_resties_mod['Borders'].fillna('NA', inplace=True)

In [670]:
colunas_com_listas = df_resties_mod.columns[df_resties_mod.applymap(type).eq(list).any()]

In [671]:
colunas_com_listas

Index(['Tld', 'Capital', 'Altspellings', 'Latlng', 'Borders', 'Timezones',
       'Continents'],
      dtype='object')

In [672]:
for colunas in colunas_com_listas:
    df_resties_mod[colunas] = df_resties_mod[colunas].apply(json.dumps)

In [673]:
df_resties_mod['Continents'] = df_resties_mod['Continents'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[2:-2])

In [674]:
df_resties_mod['Capital'] = df_resties_mod['Capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[2:-2])

In [675]:
df_resties_mod['Latlng'] = df_resties_mod['Latlng'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [676]:
df_resties_mod['Borders'] = df_resties_mod['Borders'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [677]:
df_resties_mod['Timezones'] = df_resties_mod['Timezones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [678]:
df_resties_mod['Capital'] = df_resties_mod['Capital'].str.strip('"')

In [679]:
df_resties_mod = df_resties_mod.drop(columns=['Translations','Demonyms','Languages','Gini','Capitalinfo','Idd','Car','Tld','Altspellings','Maps','Flags','Coatofarms'])

In [680]:

def converter_dict_para_list(value):
    if isinstance(value, dict):
        return list(value)  # Converte o dicionário em uma string
    else:
        return value  # Mantém outros valores inalterados


In [681]:
#todas_as_colunas_dict = ['Translations']

In [682]:
#for colunas in todas_as_colunas_dict:
#    df_resties_mod[colunas] = df_resties_mod[colunas].apply(converter_dict_para_list)


In [683]:
#df_resties_mod['Translations'] = df_resties_mod['Translations'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [684]:
df_resties_mod['Timezones'] = df_resties_mod['Timezones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [685]:
df_resties_mod.head(2)

,Name,Cca2,Ccn3,Cca3,Cioc,Independent,Status,Unmember,Currencies_Name,Capital,Region,Subregion,Latlng,Landlocked,Borders,Area,Flag,Population,Fifa,Timezones,Continents,Startofweek
0,Palestine,PS,275,PSE,PLE,False,officially-assigned,False,Egyptian pound,"Ramallah"", ""Jerusalem",Asia,Western Asia,"31.9, 35.2",False,"""ISR"", ""EGY"", ""JOR""",6220.0,🇵🇸,4803269,PLE,UTC+02:00,Asia,sunday
1,Kuwait,KW,414,KWT,KUW,True,officially-assigned,True,Kuwaiti dinar,Kuwait City,Asia,Western Asia,"29.5, 45.75",False,"""IRQ"", ""SAU""",17818.0,🇰🇼,4270563,KUW,UTC+03:00,Asia,sunday


In [686]:
df_resties_mod['Cioc'] = df_resties_mod['Cioc'].fillna('Not_Apply')

In [687]:
valores_ausentes_por_coluna = df_resties_mod.isna().sum()

In [688]:
valores_ausentes_por_coluna

Name                0
Cca2                0
Ccn3                1
Cca3                0
Cioc                0
Independent         1
Status              0
Unmember            0
Currencies_Name     3
Capital             0
Region              0
Subregion           5
Latlng              0
Landlocked          0
Borders             0
Area                0
Flag                0
Population          0
Fifa               44
Timezones           0
Continents          0
Startofweek         0
dtype: int64

In [689]:
df_resties_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             250 non-null    object 
 1   Cca2             250 non-null    object 
 2   Ccn3             249 non-null    object 
 3   Cca3             250 non-null    object 
 4   Cioc             250 non-null    object 
 5   Independent      249 non-null    object 
 6   Status           250 non-null    object 
 7   Unmember         250 non-null    bool   
 8   Currencies_Name  247 non-null    object 
 9   Capital          250 non-null    object 
 10  Region           250 non-null    object 
 11  Subregion        245 non-null    object 
 12  Latlng           250 non-null    object 
 13  Landlocked       250 non-null    bool   
 14  Borders          250 non-null    object 
 15  Area             250 non-null    float64
 16  Flag             250 non-null    object 
 17  Population      

In [691]:
mapeamento = {True: 'Yes', False: 'No'}

In [692]:
colunas_a_tratar = ['Independent', 'Unmember']
df_resties_mod[colunas_a_tratar] = df_resties_mod[colunas_a_tratar].replace(mapeamento)

In [697]:
df_resties_mod.head(2)

,Name,Cca2,Ccn3,Cca3,Cioc,Independent,Status,Unmember,Currencies_Name,Capital,Region,Subregion,Latlng,Landlocked,Borders,Area,Flag,Population,Fifa,Timezones,Continents,Startofweek
0,Palestine,PS,275,PSE,PLE,No,officially-assigned,No,Egyptian pound,"Ramallah"", ""Jerusalem",Asia,Western Asia,"31.9, 35.2",False,"""ISR"", ""EGY"", ""JOR""",6220.0,🇵🇸,4803269,PLE,UTC+02:00,Asia,sunday
1,Kuwait,KW,414,KWT,KUW,Yes,officially-assigned,Yes,Kuwaiti dinar,Kuwait City,Asia,Western Asia,"29.5, 45.75",False,"""IRQ"", ""SAU""",17818.0,🇰🇼,4270563,KUW,UTC+03:00,Asia,sunday


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [693]:
#Não sei por qual motivo o df_resties_mod da falha na hora de ser enviado para o db, e quando vai ele retorna somente com as colunas.

salva_bd(df_resties_mod,'resties')

In [694]:
lista_tabelas()

,name
0,produtos
1,restaurantes
2,fastfood
3,bancos
4,corretoras
5,resties


In [695]:
resties_teste = carrega_bd('resties')

In [696]:
resties_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             250 non-null    object 
 1   Cca2             250 non-null    object 
 2   Ccn3             249 non-null    object 
 3   Cca3             250 non-null    object 
 4   Cioc             250 non-null    object 
 5   Independent      249 non-null    object 
 6   Status           250 non-null    object 
 7   Unmember         250 non-null    object 
 8   Currencies_Name  247 non-null    object 
 9   Capital          250 non-null    object 
 10  Region           250 non-null    object 
 11  Subregion        245 non-null    object 
 12  Latlng           250 non-null    object 
 13  Landlocked       250 non-null    int64  
 14  Borders          250 non-null    object 
 15  Area             250 non-null    float64
 16  Flag             250 non-null    object 
 17  Population      